<a href="https://colab.research.google.com/github/kazuto1919/20241013-git-handson/blob/main/generation_by_source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from google.colab import drive
drive.mount('/content/drive')

from openpyxl import Workbook
from openpyxl import load_workbook
from glob import glob

now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [97]:
# インプットディレクトとアウトプットディレクトリの作成

os.chdir('/content/drive/MyDrive/electrica/data_analysis/1_tso')
current_dir = os.getcwd()
input_dir =os.path.join(current_dir, 'input')
output_dir = os.path.join(current_dir, 'output')



#2024年以前のデータ処理　基本的に一度だけ回す

In [ ]:
# 四国を除く、各エリアのcsvファイルの取得（四国はexcelファイルのため）

areas = ['hepco', 'tohoku', 'tepco', 'chubu', 'hokuriku', 'kepco', 'chugoku','kyushu']
area_files = {}
for area in areas:
    file_pattern = os.path.join(input_dir, area + '*pre.csv')
    area_files[area] = glob(file_pattern)

# 四国エリアのexcelファイルの取得

shikoku_pattern = os.path.join(input_dir, 'shikoku*.xlsx')
area_files['shikoku'] = glob(shikoku_pattern)

In [146]:
#コラムが統一される前の各エリアのコラム名の設定

hokkaido_columns_before2024 = ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector', 'total_generation')
tohoku_columns_before2024 = ('datetime','total_demand', 'hydro', 'fuel', 'nuclear', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'geothermal', 'biomass', 'pumped_storage', 'interconnector')
tokyo_columns_before2024 = ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector', 'total_generation')
chubu_columns_before2024 =  ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector')
hokuriku_columns_before2024 = ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector')
kepco_columns_before2024 = ('datetime','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector')
chugoku_columns_before2024 = ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector')
shikoku_columns_before2024 = ('date','time','total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector', 'total_generation')
kyushu_columns_before2024 = ('datetime', 'total_demand', 'nuclear', 'fuel', 'hydro', 'geothermal', 'biomass', 'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'interconnector')

In [147]:
"""

以下各エリアの2024年以前のコラム名が統一される前のデータの集計
エリアによって、日付のフォーマットや空白行数が異なるため、繰り返し動作にはしていない

"""

# 北海道エリア

data_frames = []

for file in area_files['hepco']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=3)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=3)
  data_frames.append(df)
  df.columns = hokkaido_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = merged_df['date'].ffill()
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['time'] = merged_df['time'].str.replace('時', '')

output_files = os.path.join(output_dir, f'hokkaido_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

# 東北エリア

data_frames = []

for file in area_files['tohoku']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS')
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8')
  data_frames.append(df)
  df.columns = tohoku_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['datetime'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['time'] = merged_df['date'].dt.time

output_files = os.path.join(output_dir, f'tohoku_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#東京エリア

data_frames = []

for file in area_files['tepco']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=2)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=2)
  data_frames.append(df)
  df.columns = tokyo_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day

output_files = os.path.join(output_dir, f'tokyo_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

# 中部エリア

data_frames = []

for file in area_files['chubu']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=4)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=4)
  data_frames.append(df)
  df.columns = chubu_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day

output_files = os.path.join(output_dir, f'chubu_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#北陸エリア

data_frames = []

for file in area_files['hokuriku']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=5)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=5)
  data_frames.append(df)
  df.columns = hokuriku_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day

output_files = os.path.join(output_dir, f'hokuriku_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#関西エリア

data_frames = []

for file in area_files['kepco']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=1)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=1)
  data_frames.append(df)
  df.columns = kepco_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['datetime'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['time'] = merged_df['date'].dt.time

output_files = os.path.join(output_dir, f'kansai_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#中国エリア

data_frames = []

for file in area_files['chugoku']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=2)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=2)
  data_frames.append(df)
  df.columns = chugoku_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day

output_files = os.path.join(output_dir, f'chugoku_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#四国エリア

data_frames = []

for file in area_files['shikoku']:
    df = pd.read_excel(file)
    data_frames.append(df)
    df.columns = shikoku_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['date'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day

output_files = os.path.join(output_dir, f'shikoku_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')

#九州エリア

data_frames = []

for file in area_files['kyushu']:
  try:
    df = pd.read_csv(file, encoding='shift-JIS', skiprows=1)
  except UnicodeDecodeError:
    df = pd.read_csv(file, encoding='utf-8', skiprows=1)
  data_frames.append(df)
  df.columns = kyushu_columns_before2024

merged_df = pd.concat(data_frames, ignore_index=True)
merged_df['date'] = pd.to_datetime(merged_df['datetime'])
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['time'] = merged_df['date'].dt.time

output_files = os.path.join(output_dir, f'kyushu_old_total_{timestamp}.csv')
merged_df.to_csv(output_files, index=False, encoding='utf-8')



```
# これはコードとして書式設定されます
```

#2024年以降のデータ処理　ファイルが更新されるごとに回す

In [152]:
#各エリアのcsvファイルの取得

hepcofiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_01.csv')
tohokufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_02.csv')
tepcofiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_03.csv')
chubufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_04.csv')
hokurikufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_05.csv')
kepcofiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_06.csv')
chugokufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_07.csv')
shikokufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_08.csv')
kyushufiles = glob('/content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_' + '*_09.csv')

In [153]:
# コラム名の設定　各エリア共通

columns = ('date','time','total_demand', 'nuclear', 'lng', 'coal', 'oil', 'other_fuel',  'hydro', 'geothermal', 'biomass',
                  'pv', 'pv_curtailment', 'wind', 'wind_curtailment', 'pumped_storage', 'bess', 'interconnector', 'other', 'total_generation')

In [155]:
# データの集計とアウトプットファイル保存の関数定義

def process_and_save(files, area_name):
  data_frames = []

  for file in files:
      try:
          df = pd.read_csv(file, encoding='shift-JIS', skiprows=1)
      except UnicodeDecodeError:
          df = pd.read_csv(file, encoding='utf-8', skiprows=1)
      data_frames.append(df)
      df.columns = columns

  merged_df = pd.concat(data_frames, ignore_index=True)
  merged_df['date'] = pd.to_datetime(merged_df['date'])
  merged_df['year'] = merged_df['date'].dt.year
  merged_df['month'] = merged_df['date'].dt.month
  merged_df['day'] = merged_df['date'].dt.day

  output_files = os.path.join(output_dir, f'{area_name}_total_{timestamp}.csv')
  merged_df.to_csv(output_files, index=False, encoding='utf-8')

In [ ]:
# 中国エリア以外のデータ集計操作、中国エリアはいくつかのファイルでコラム数が違うため別処理

process_and_save(hepcofiles, 'hepco')
process_and_save(tohokufiles, 'tohoku')
process_and_save(tepcofiles, 'tepco')
process_and_save(chubufiles, 'chubu')
process_and_save(hokurikufiles, 'hokuriku')
process_and_save(kepcofiles, 'kansai')
process_and_save(shikokufiles, 'shikoku')
process_and_save(kyushufiles, 'kyushu')

In [157]:
# 中国エリアインプットファイル内のコラム数　確認

for file in chugokufiles:
    try:
        df = pd.read_csv(file, encoding='shift-JIS', skiprows=1)
    except UnicodeDecodeError:
        df = pd.read_csv(file, encoding='utf-8', skiprows=1)
    num_cols = len(df.columns)
    print(f"File: {file}, Number of columns: {num_cols}")

File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202404_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202405_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202406_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202407_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202408_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202409_07.csv, Number of columns: 20
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202410_07.csv, Number of columns: 22
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202411_07.csv, Number of columns: 22
File: /content/drive/MyDrive/electrica/data_analysis/1_tso/input/eria_jukyu_202412_07.csv, Numbe

# 2024年以前以後のデータの統合

In [148]:
#　2024年以前の集計後ファイルの読み込み

hokkaido_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/hokkaido_old_total_20250217.csv')
tohoku_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/tohoku_old_total_20250217.csv')
tokyo_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/tokyo_old_total_20250217.csv')
chubu_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/chubu_old_total_20250217.csv')
hokuriku_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/hokuriku_old_total_20250217.csv')
kansai_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/kansai_old_total_20250217.csv')
chugoku_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/chugoku_old_total_20250217.csv')
shikoku_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/shikoku_old_total_20250217.csv')
kyushu_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/kyushu_old_total_20250217.csv')

<ipython-input-148-1a01a214c0e2>:3: DtypeWarning: Columns (2,4,14) have mixed types. Specify dtype option on import or set low_memory=False.
  tokyo_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/tokyo_old_total_20250217.csv')
<ipython-input-148-1a01a214c0e2>:5: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  hokuriku_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/hokuriku_old_total_20250217.csv')
<ipython-input-148-1a01a214c0e2>:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  kansai_old_total = pd.read_csv('/content/drive/MyDrive/electrica/data_analysis/1_tso/output/kansai_old_total_20250217.csv')
<ipython-input-148-1a01a214c0e2>:7: DtypeWarning: Columns (0,1,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  chugoku_old_total = pd.read_csv('/co

In [151]:
# 単位1万kWのエリアを1MWに合わせる

# 北海道エリア

hokkaido_old_total_adj = hokkaido_old_total.loc[:, 'total_demand':'total_generation'] * 10
hokkaido_old_total_adj['date'] = hokkaido_old_total['date']
hokkaido_old_total_adj['time'] = hokkaido_old_total['time']
hokkaido_old_total_adj['year'] = hokkaido_old_total['year']
hokkaido_old_total_adj['month'] = hokkaido_old_total['month']
hokkaido_old_total_adj['day'] = hokkaido_old_total['day']

#東京エリア

tokyo_old_total_adj = tokyo_old_total.loc[:, 'total_demand':'total_generation'] * 10
tokyo_old_total_adj['date'] = tokyo_old_total['date']
tokyo_old_total_adj['time'] = tokyo_old_total['time']
tokyo_old_total_adj['year'] = tokyo_old_total['year']
tokyo_old_total_adj['month'] = tokyo_old_total['month']
tokyo_old_total_adj['day'] = tokyo_old_total['day']

#四国エリア

shikoku_old_total_adj = shikoku_old_total.loc[:, 'total_demand':'total_generation'] * 10
shikoku_old_total_adj['date'] = shikoku_old_total['date']
shikoku_old_total_adj['time'] = shikoku_old_total['time']
shikoku_old_total_adj['year'] = shikoku_old_total['year']
shikoku_old_total_adj['month'] = shikoku_old_total['month']
shikoku_old_total_adj['day'] = shikoku_old_total['day']